# ¿Qué es Cython?

Cython son ***dos*** cosas:

* Por una parte, Cython es un lenguaje de programación (un ***superconjunto*** de Python) que une Python con el sistema de ***tipado estático*** de C y C++.
* Por otra parte, `cython` es un ***compilador*** que traduce codigo fuente escrito en Cython en eficiente código ***C o C++***. El código resultante se podría usar como una extensión Python o como un ejecutable.

# Documentación

La página oficial del proyecto la podéis encontrar en [cython.org](http://cython.org/). Desde ahí podéis llegar a varios recursos de la [documentación oficial y no oficial](http://cython.org/#documentation).

En general, la documentación oficial está bien y es suficiente para empezar a aprender sobre cython y aplicarlo inmediatamente. En algunas secciones está un poco desactualizado pero no es mayor problema.

### Otros recursos para aprender

* [Learning Cython Programming](https://www.packtpub.com/application-development/learning-cython-programming)
<hr><center><img src="./static/Learning Cython Programming.jpg"/></center><hr>

* [Cython, a guide for Python programmers](http://shop.oreilly.com/product/0636920033431.do)
<hr><center><img src="./static/Cython_oreilly.jpg"/ width=300 height=350></center><hr>

* [Vídeos](http://pyvideo.org/search?models=videos.video&q=cython)
 * Recomendaría el [tutorial de Scipy 2013](http://pyvideo.org/video/2162/cython-speed-up-python-and-numpy-pythonize-c-c-7) como el más completo.

# Brevísima introducción a algunas características de Cython

Lo que se pretende es, básicamente, aprovechar las fortalezas de Python y C, combinar una sintaxis sencilla con el poder y la velocidad.

Salvando algunas [excepciones](http://docs.cython.org/src/userguide/limitations.html#cython-limitations), el código Python (tanto Python 2 como Python 3) es código Cython válido. Además, Cython añade una serie de palabras clave para poder usar el sistema de tipado de C con Python y que el compilador `cython` pueda generar código C eficiente.

Pero, ¿quién usa Cython?

Pues mira, igual no lo sabes pero seguramente estés usando Cython todos los días. Sage tiene casi medio millón de líneas de Cython (que se dice pronto), Scipy y Pandas más de 20000, scikit-learn unas 15000,...

# El problema que vamos a optimizar

Deflexión de una placa rectangular, simplemente apoyada en sus cuatro bordes (es decir, los bordes pueden girar: no están empotrados) sometida a una carga transversal. Este problema tiene solución analítica conocida desde hace tiempo, hallada por Navier:

$$w(x,y) = \sum_{m=1}^\infty \sum_{n=1}^\infty \frac{a_{mn}}{\pi^4 D}\,\left(\frac{m^2}{a^2}+\frac{n^2}{b^2}\right)^{-2}\,\sin\frac{m \pi x}{a}\sin\frac{n \pi y}{b}$$

siendo $a_{mn}$ los coeficientes de Fourier de la carga aplicada. Como veis, para cada punto $(x, y)$ hay que hacer una doble suma en serie; si encima queremos evaluar esto en un `meshgrid`, necesitamos **un cuádruple bucle**. Ya se anticipa que por muy hábiles que estemos, a Python le va a costar.

<DIV class="alert alert-info">Los números y porcentajes que veáis a continuación pueden variar levemente dependiendo de la máquina donde se ejecute. Tomad los valores como aproximados.</div>

# Preparando el problema

Importamos algunas librerías

In [1]:
import numpy as np
from math import pi, sin
import matplotlib.pyplot as plt
%matplotlib inline

La versión que vamos a considerar como la estándar y que vamos a optimizar:

In [2]:
def plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    max_i, max_j = ww.shape
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii, jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii, jj] / a)
                                   * sin(nn * pi * yy[ii, jj] / b)
                                   / (pi**4 * D)) 

Definimos un caso de ejemplo:

In [3]:
# Plate geometry
a = 1.0  # m
b = 1.0  # m
h = 50e-3  # m

# Material properties
E = 69e9  # Pa
nu = 0.35

# Series terms
max_m = 16
max_n = 16

# Computation points
# NOTE: With an odd number of points the center of the place is included in
# the grid
NUM_POINTS = 101

# Load
P = 10e3  # N
xi = a / 2
eta = a / 2

# Flexural rigidity
D = h**3 * E / (12 * (1 - nu**2))

# ---

# Set up domain
x = np.linspace(0, a, num=NUM_POINTS)
y = np.linspace(0, b, num=NUM_POINTS)
xx, yy = np.meshgrid(x, y)

ww = np.zeros_like(xx)

Veamos el resultado:

In [4]:
# Compute displacement field
plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

# Print maximum displacement
w_max = abs(ww).max()
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))

Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm


Y el tiempo de ejecución:

In [5]:
%timeit plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

1 loops, best of 3: 1min 1s per loop


# Cython (prueba 1)

Lo más sencillo y evidente es usar directamente el compilador `cython` y ver si usando el código python tal cual es un poco más rápido. Para ello, vamos a usar las funciones mágicas que Cython pone a nuestra disposición  en el notebook. Solo vamos a hablar de la función mágica `%%cython`, de momento, aunque hay otras.

In [1]:
# antes cythonmagic
%load_ext Cython

El comando `%%cython` nos permite escribir código Cython en una celda. Una vez que ejecutamos la celda, IPython se encarga de coger el código, crear un fichero de código Cython con extensión *.pyx*, compilarlo a C y, si todo está correcto, importar ese fichero para que todo esté disponible dentro del notebook.

<div class="alert alert-info">A la función mágica `%%cython` le podemos pasar una serie de argumentos. Veremos alguno en este análisis pero ahora vamos a definir uno que sirve para que podamos nombrar a la funcíon que se crea y compila al vuelo, `-n` o `--name`.</div>

In [11]:
%%cython --name probandocython1
import numpy as np
from numpy import sin, pi

def plate_displacement_cy1(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    max_i, max_j = ww.shape
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii, jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii, jj] / a)
                                   * sin(nn * pi * yy[ii, jj] / b)
                                   / (pi**4 * D)) 

El fichero se creará dentro de la carpeta *cython* disponible dentro del directorio resultado de la función `get_ipython_cache_dir`. Veamos la localización del fichero en mi equipo:

In [12]:
from IPython.utils.path import get_ipython_cache_dir

In [13]:
print(get_ipython_cache_dir() + '/cython/probandocython1.c')

/home/kiko/.cache/ipython/cython/probandocython1.c


Veamos lo que es un fichero compilado a C con Cython:

In [ ]:
%load /home/kiko/.cache/ipython/cython/probandocython1.c

Y veamos el tiempo de ejecución:

In [15]:
%timeit plate_displacement_cy1(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

1 loops, best of 3: 2min 15s per loop


In [17]:
171 / 135 # Segundos

1.2666666666666666

Parece que, sin hacer ningún cambio al código, ya hemos conseguido alrededor de un 25% mejora del rendimiento.

Pero cython  puede hacer mucho más.

# Cython (prueba 2)

En esta parte vamos a introducir una de las palabras clave que Cython introduce para extender Python, `cdef`. La palabra clave `cdef` sirve para 'tipar' estáticamente variables en Cython (luego veremos que se usa también para definir funciones). Por ejemplo:

```Python
cdef int var1, var2
cdef float var3
```

En el bloque de código de más arriba he creado dos variables de tipo entero, `var1` y `var2`, y una variable de tipo float, `var3`. Los [tipos anteriores son la nomenclatura C](http://docs.cython.org/src/userguide/language_basics.html#automatic-type-conversions).

<div class="alert alert-info">En esta charla nos vamos a centrar en los <a href="https://en.wikipedia.org/wiki/C_data_types#Basic_types">tipos básicos de C que podéis ver resumidos en la siguiente tabla de referencia.</a></div>

<table class="wikitable">
<tbody><tr>
<th style="width:14em;">Type</th>
<th>Explanation</th>
<th>Format Specifier</th>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">char</span></span></td>
<td>Smallest addressable unit of the machine that can contain basic character set. It is an integer type. Actual type can be either signed or unsigned depending on the implementation.</td>
<td>&nbsp;%c</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">signed</span> <span class="kw4">char</span></span></td>
<td>Of the same size as <code>char</code>, but guaranteed to be signed.</td>
<td>&nbsp;%c</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">unsigned</span> <span class="kw4">char</span></span></td>
<td>Of the same size as <code>char</code>, but guaranteed to be unsigned.</td>
<td>&nbsp;%c</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">short</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">short</span> <span class="kw4">int</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">signed</span> <span class="kw4">short</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">signed</span> <span class="kw4">short</span> <span class="kw4">int</span></span></td>
<td><i>Short</i> signed integer type. Capable of containing at least the [−32767,+32767] range;<sup id="cite_ref-c99sizes_3-0" class="reference"><a href="#cite_note-c99sizes-3"><span>[</span>3<span>]</span></a></sup> thus, it is at least 16 bits in size.</td>
<td>&nbsp;%hi</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">unsigned</span> <span class="kw4">short</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">unsigned</span> <span class="kw4">short</span> <span class="kw4">int</span></span></td>
<td>The same as <code>short</code>, but unsigned.</td>
<td>&nbsp;%hu</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">int</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">signed</span> <span class="kw4">int</span></span></td>
<td>Basic signed integer type. Capable of containing at least the [−32767,+32767] range;<sup id="cite_ref-c99sizes_3-1" class="reference"><a href="#cite_note-c99sizes-3"><span>[</span>3<span>]</span></a></sup> thus, it is at least 16 bits in size.</td>
<td>&nbsp;%i or&nbsp;%d</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">unsigned</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">unsigned</span> <span class="kw4">int</span></span></td>
<td>The same as <code>int</code>, but unsigned.</td>
<td>&nbsp;%u</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">long</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">long</span> <span class="kw4">int</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">signed</span> <span class="kw4">long</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">signed</span> <span class="kw4">long</span> <span class="kw4">int</span></span></td>
<td><i>Long</i> signed integer type. Capable of containing at least the [−2147483647,+2147483647] range;<sup id="cite_ref-c99sizes_3-2" class="reference"><a href="#cite_note-c99sizes-3"><span>[</span>3<span>]</span></a></sup> thus, it is at least 32 bits in size.</td>
<td>&nbsp;%li</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">unsigned</span> <span class="kw4">long</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">unsigned</span> <span class="kw4">long</span> <span class="kw4">int</span></span></td>
<td>The same as <code>long</code>, but unsigned.</td>
<td>&nbsp;%lu</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">long</span> <span class="kw4">long</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">long</span> <span class="kw4">long</span> <span class="kw4">int</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">signed</span> <span class="kw4">long</span> <span class="kw4">long</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">signed</span> <span class="kw4">long</span> <span class="kw4">long</span> <span class="kw4">int</span></span></td>
<td><i>Long long</i> signed integer type. Capable of containing at least the [−9223372036854775807,+9223372036854775807] range;<sup id="cite_ref-c99sizes_3-3" class="reference"><a href="#cite_note-c99sizes-3"><span>[</span>3<span>]</span></a></sup> thus, it is at least 64 bits in size. Specified since the <a href="/wiki/C99" title="C99">C99</a> version of the standard.</td>
<td>&nbsp;%lli</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">unsigned</span> <span class="kw4">long</span> <span class="kw4">long</span></span><br>
<span class="mw-geshi cpp source-cpp"><span class="kw4">unsigned</span> <span class="kw4">long</span> <span class="kw4">long</span> <span class="kw4">int</span></span></td>
<td>The same as <code>long long</code>, but unsigned. Specified since the <a href="/wiki/C99" title="C99">C99</a> version of the standard.</td>
<td>&nbsp;%llu</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">float</span></span></td>
<td>Real floating-point type, usually referred to as a single-precision floating-point type. Actual properties unspecified (except minimum limits), however on most systems this is the <a href="/wiki/Single-precision_floating-point_format" title="Single-precision floating-point format">IEEE 754 single-precision binary floating-point format</a>. This format is required by the optional Annex F "IEC 60559 floating-point arithmetic".</td>
<td>&nbsp;%f</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">double</span></span></td>
<td>Real floating-point type, usually referred to as a double-precision floating-point type. Actual properties unspecified (except minimum limits), however on most systems this is the <a href="/wiki/Double-precision_floating-point_format" title="Double-precision floating-point format">IEEE 754 double-precision binary floating-point format</a>. This format is required by the optional Annex F "IEC 60559 floating-point arithmetic".</td>
<td>&nbsp;%f</td>
</tr>
<tr>
<td><span class="mw-geshi cpp source-cpp"><span class="kw4">long</span> <span class="kw4">double</span></span></td>
<td>Real floating-point type, usually mapped to an <a href="/wiki/Extended_precision" title="Extended precision">extended precision</a> floating-point number format. Actual properties unspecified. Unlike types <span class="mw-geshi cpp source-cpp"><span class="kw4">float</span></span> and <span class="mw-geshi cpp source-cpp"><span class="kw4">double</span></span>, it can be either <a href="/wiki/80-bit_floating_point_format" title="80-bit floating point format" class="mw-redirect">80-bit floating point format</a>, the non-IEEE "<a href="/wiki/Double-double_arithmetic" title="Double-double arithmetic" class="mw-redirect">double-double</a>" or <a href="/wiki/IEEE_754_quadruple-precision_floating-point_format" title="IEEE 754 quadruple-precision floating-point format" class="mw-redirect">IEEE 754 quadruple-precision floating-point format</a> if a higher precision format is provided, otherwise it is the same as <span class="mw-geshi cpp source-cpp"><span class="kw4">double</span></span>. See <a href="/wiki/Long_double" title="Long double">the article on long double</a> for details.</td>
<td>&nbsp;%lf</td>
</tr>
</tbody></table>

Vamos a intentar usar `cdef` con algunos tipos de datos que tenemos dentro de nuestra función. Para empezar, tenemos los índices de los bucles (`mm`, `nn`, `ii` y `jj`) y voy a convertir los parámetros de los `range` en tipos estáticos (`max_m`, `max_n`, `max_i` y `max_j`):

In [18]:
%%cython --name probandocython2
import numpy as np
from numpy import sin, pi

def plate_displacement_cy2(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    cdef unsigned int max_i, max_j, mm, nn, ii, jj
    max_i, max_j = ww.shape
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii, jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii, jj] / a)
                                   * sin(nn * pi * yy[ii, jj] / b)
                                   / (pi**4 * D)) 

In [19]:
%timeit plate_displacement_cy2(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

1 loops, best of 3: 2min 29s per loop


Vaya decepción... No hemos conseguido gran cosa, tenemos un código un poco más largo y estamos peor que en la **toma 1**.

En realidad, estamos usando objetos Python como numpy arrays y no hemos definido las variables de entrada y de salida de la función.

<div class="alert alert-info">Cuando existe un tipo Python y C que tienen el mismo nombre (por ejemplo, `int`) predomina el de C (porque es lo deseable, ¿no?).</div>

# Cython (prueba 3)

En Cython existen tres tipos de funciones, las definidas en el espacio Python con `def`, las definidas en el espacio C con `cdef` (sí, lo mismo que usamos para declarar los tipos) y las definidas en ambos espacios con `cpdef`.

* `def`: ya lo hemos visto y funciona como se espera. Accesible desde Python
* `cdef`: No es accesible desde Python y la tendremos que envolver con una función Python para poder acceder a la misma.
* `cpdef`: Es accesible tanto desde Python como desde C y Cython se encargará de hacer el 'envoltorio' para nosotros. Esto meterá un poco más de código y empeorará levemente el rendimiento.

Si definimos una función con `cdef` debería ser una función que se usa internamente dentro del módulo Cython que vayamos a crear y que no sea necesario llamar desde Python.

La función mágica `%%cython` dispone de una serie de funcionalidades entre la que se encuentra `-a` o `--annotate` (además del `-n` o `--name` que ya hemos visto). Si le pasamos este parámetro podremos ver una representación del código con colores marcando las partes más lentas (amarillo más oscuro) y más optmizadas (más claro) o a la velocidad de C (blanco). Vamos a usarlo para saber donde tenemos cuellos de botella (aplicado a nuestra última versión del código):

In [20]:
%%cython --annotate
import numpy as np
from numpy import sin, pi

def plate_displacement_cy2(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    cdef unsigned int max_i, max_j, mm, nn, ii, jj
    max_i, max_j = ww.shape
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii, jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii, jj] / a)
                                   * sin(nn * pi * yy[ii, jj] / b)
                                   / (pi**4 * D)) 

Vaya, parece que no tenemos muchas partes optimizadas, aunque los `for` parece que deberían ser más eficientes ya que aparecen más claros que el resto del código.

Ahora mismo, haciendo `import numpy as np` tenemos acceso a la funcionalidad Python de numpy. Para poder acceder a la funcionalidad C de numpy hemos de hacer un `cimport` de numpy.

El `cimport` se usa para importar información especial del módulo numpy en el momento de compilación. Esta información se encuentra en el fichero numpy.pxd que es parte de la distribución Cython. El `cimport` también se usa para poder importar desde la *stdlib* de C.

Vamos a usar esto para declarar el tipo del array de numpy.

In [2]:
%%cython --name probandocython3
import numpy as np
cimport numpy as np
from numpy import sin, pi

def plate_displacement_cy3(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    cdef unsigned int max_i, max_j, mm, nn, ii, jj
    max_i, max_j = ww.shape
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii, jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii, jj] / a)
                                   * sin(nn * pi * yy[ii, jj] / b)
                                   / (pi**4 * D)) 

In [7]:
%timeit plate_displacement_cy3(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

1 loops, best of 3: 2min 35s per loop


Seguimos sin estar muy contentos con este resultado. Sigamos iterando.

# Cython (prueba 4)

Podemos usar [directivas de compilación](http://docs.cython.org/src/reference/compilation.html#compiler-directives) que ayuden al compilador a decidir mejor qué es lo que tiene que hacer. Entre ellas se encuentra una opción que es `boundscheck` que evita mirar la posibilidad de obtener `IndexError` asumiendo que el código está libre de estos errores de indexación. Lo vamos a usar conjuntamente con `wraparound`. Esta última opción se encarga de evitar mirar indexaciones relativas al final del iterable (por ejemplo, `mi_iterable[-1]`). En este caso concreto, la segunda opción no aporta nada de mejora de rendimiento pero la dijamos ya que la hemos probado.

Estas ***directivas de compilación*** son muy flexibles ya quee las podemos aplicar de forma global, solo a ciertas partes del código,... En este caso las voy a usar como decoradores de la función por lo que solo se aplican a la función:

In [3]:
%%cython --name probandocython4
import numpy as np
cimport numpy as np
from numpy import sin, pi
cimport cython

@cython.boundscheck(False) 
@cython.wraparound(False)
def plate_displacement_cy4(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    cdef unsigned int max_i, max_j, mm, nn, ii, jj
    max_i, max_j = ww.shape
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii, jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii, jj] / a)
                                   * sin(nn * pi * yy[ii, jj] / b)
                                   / (pi**4 * D)) 

Y miramos el rendimiento con estas nuevas modificaciones:

In [7]:
%timeit plate_displacement_cy4(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

1 loops, best of 3: 2min 28s per loop
